In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt


# Getting the dataset and converting into dataframe

In [2]:
dataset = files.upload()


Saving diabetes.csv to diabetes.csv


In [3]:
dataframe = pd.read_csv('/content/diabetes.csv')

In [4]:
dataframe.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Defining features and target Variable

In [5]:
from keras.models import Sequential
from keras.layers import Dense
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
random.seed(42)

y = dataframe.Outcome
x = dataframe.drop('Outcome',axis=1)
#x = dataframe.iloc[:,0:8]
#y = dataframe.iloc[:,8]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.33,random_state=1)

# Define the Model architecture

In [6]:
model = Sequential()
model.add(Dense(12,input_dim=8,  activation= 'relu' ))
model.add(Dense(8,  activation= 'relu' ))
model.add(Dense(1,  activation= 'sigmoid' ))

In [7]:
model.compile(loss='binary_crossentropy',optimizer= 'adam', metrics=['accuracy']) #configuration for training the model 

In [8]:
model.fit(x,y,batch_size=20,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
39/39 [==============================] - 6s 13ms/step - loss: 4.7663 - accuracy: 0.4727 - val_loss: 3.6272 - val_accuracy: 0.4213
Epoch 2/100
39/39 [==============================] - 0s 11ms/step - loss: 2.7399 - accuracy: 0.4648 - val_loss: 2.1267 - val_accuracy: 0.4803
Epoch 3/100
39/39 [==============================] - 0s 10ms/step - loss: 1.8918 - accuracy: 0.4883 - val_loss: 1.5488 - val_accuracy: 0.4882
Epoch 4/100
39/39 [==============================] - 0s 9ms/step - loss: 1.4536 - accuracy: 0.5234 - val_loss: 1.2219 - val_accuracy: 0.5354
Epoch 5/100
39/39 [==============================] - 0s 7ms/step - loss: 1.1896 - accuracy: 0.5534 - val_loss: 1.0198 - val_accuracy: 0.5630
Epoch 6/100
39/39 [==============================] - 0s 8ms/step - loss: 1.0171 - accuracy: 0.5677 - val_loss: 0.9356 - val_accuracy: 0.5827
Epoch 7/100
39/39 [==============================] - 0s 8ms/step - loss: 0.9319 - accuracy: 0.5859 - val_loss: 0.8807 - val_accuracy: 0.6024
Epoch 8/10

In [9]:
scores = model.evaluate(x,y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.5234 - accuracy: 0.7500
accuracy: 75.00%


# K-Fold Validation Sets

In [10]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=1)
cross_val = []
for train_index, test_index in kfold.split(x, y):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = Sequential()
    model.add(Dense(12,input_dim=8,  activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    model.compile(loss='binary_crossentropy',optimizer= 'adam', metrics=['accuracy']) #configuration for training the model 
    model.fit(X_train,y_train,batch_size=20,epochs=150,verbose = 0)
    scores = model.evaluate(X_test,y_test,verbose=0)
    print("%s : %.2f%%"%(model.metrics_names[1],scores[1]*100))
    cross_val.append(scores[1]*100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cross_val), np.std(cross_val)))

accuracy : 70.13%
accuracy : 72.73%
accuracy : 68.83%


accuracy : 71.43%


accuracy : 75.32%
accuracy : 76.62%
accuracy : 64.94%
accuracy : 67.53%
accuracy : 69.74%
accuracy : 60.53%
69.78% (+/- 4.52%)


# Keras with Scikit Learn

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np

def create_model():
    model = Sequential()
    model.add(Dense(12,input_dim=8,  activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    model.compile(loss='binary_crossentropy',optimizer= 'adam', metrics=['accuracy']) #configuration for training the model 
    return model

seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model,nb_epoch=150, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

<ipython-input-11-4de2de61e123>:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model,nb_epoch=150, batch_size=10, verbose=0)


0.5159261852502823


In [12]:
from keras.backend import batch_flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from keras.backend import batch_flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

def create_model(optimizer='rmsprop', kernel_initializer='glorot_uniform'):
  model = Sequential()
  model.add(Dense(12, input_dim=8, kernel_initializer=kernel_initializer, activation='relu'))
  model.add(Dense(8, kernel_initializer=kernel_initializer, activation='relu'))
  model.add(Dense(1, kernel_initializer=kernel_initializer, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
kernel_initializer = ['glorot_uniform', 'normal', 'uniform']
epochs = np.array([50, 75,100])
batches = np.array([5, 10,20])
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, kernel_initializer=kernel_initializer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, std_score in zip(grid_result.cv_results_['params'], 
                                         grid_result.cv_results_['mean_test_score'],
                                         grid_result.cv_results_['std_test_score']):
    print("%f (%f) with: %r" % (mean_score, std_score, params))


<ipython-input-12-25f4be7035f8>:24: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.754036 using {'batch_size': 5, 'epochs': 75, 'kernel_initializer': 'uniform', 'optimizer': 'rmsprop'}
0.665283 (0.046665) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.721399 (0.040159) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'glorot_uniform', 'optimizer': 'adam'}
0.708412 (0.040567) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'normal', 'optimizer': 'rmsprop'}
0.724005 (0.028277) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'normal', 'optimizer': 'adam'}
0.717435 (0.050945) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'uniform', 'optimizer': 'rmsprop'}
0.726551 (0.012560) with: {'batch_size': 5, 'epochs': 50, 'kernel_initializer': 'uniform', 'optimizer': 'adam'}
0.691410 (0.017678) with: {'batch_size': 5, 'epochs': 75, 'kernel_initializer': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.690298 (0.061628) with: {'batch_size': 5, 'epochs': 75, 'kernel_initializ